In [1]:
from pyspark import SparkConf, SparkContext

def mapper1(line):
    matrix = ''
    i, j = '', ''
    value = ''
    tmp = 0
    idone = False
    jdone = False
    
    for ch in line:
        if ch == 'M':
            matrix = 'M'
        elif ch == 'N':
            matrix = 'N'
        elif ch.isdigit():
            if i == '' or idone == False:
                i += ch
            elif j == '' or jdone == False:
                j += ch
            else:
                value += ch
        else:
            if tmp == 1:
                idone = True
            elif tmp == 2:
                jdone = True
            tmp+=1
                
    if matrix == 'M':
        return (j, (matrix, i, value))
    else:
        return (i, (matrix, j, value))
    
def mapper2(tp):
    i = int(tp[1][0][1])
    k = int(tp[1][1][1])
    Mnum = int(tp[1][0][2]) 
    Nnum = int(tp[1][1][2])
    value = Mnum * Nnum
    return ((i, k), value)

file = sc.textFile("500input.txt")
matrixM = file.filter(lambda x: "M" in x)
matrixN = file.filter(lambda x: "N" in x)

newM = matrixM.map(mapper1)
newN = matrixN.map(mapper1)

newmatrix = newM.join(newN)
#print("Done join")
newmatrix = newmatrix.map(mapper2)
#print("Done map1")
newmatrix = newmatrix.reduceByKey(lambda a, b: a+b)
#print("Done reduce")
newmatrix = newmatrix.sortByKey(ascending = True)
#print("Done sort")
newmatrix = newmatrix.collect()
#print("Done collect")


#print("start output")
outF = open("Outputfile.txt", "w")
for line in newmatrix:
    outF.write((str(line[0][0]) + ',' + str(line[0][1]) + ',' + str(line[1])))
    outF.write("\n")
outF.close()
    

# for item in newmatrix:
#     print(item[0][0] + ',' + item[0][1] + ',' + str(item[1]))
    
sc.stop()


Done join
Done map1
Done reduce
Done sort
Done collect
start output


### 107062321_HW1_report
這次的作業要實作兩個大矩陣的乘法，透過MapReduce的理論概念來完成

### 資料前處理

一開始丟進來的資料會有$MN$矩陣混雜在一起

我透過RDD filter的操作，將$MN$矩陣的資料分開


### mapper1

+ $MN$乘出來的矩陣$P$，其中每一個$P_{ik}$項的運算過程為$P_{ij} = \sum M_{ik}N_{kj}$

從這個概念，我做了第一個Map function

由於不能使用pylist，所以我就把每一行txt檔的文字用for迴圈掃過一次，寫一些判斷式，即可把需要的資訊抓下來。

把需要的資訊拿到後，要回傳的 key vlaue pair的形式為 :

+ 若為$M$矩陣，回傳 ( j, (matrix, i, value) )，j 代表$M$矩陣的哪個column，matrix代表是$M$或$N$，i 代表$M$矩陣的哪個row，value則代表$M_{ij}$的值

+ 若為$N$矩陣，回傳 ( i, (matrix, j, value) )，j 代表$N$矩陣的哪個row，matrix代表是$M$或$N$，i 代表$N$矩陣的哪個column，value則代表$N_{ji}$的值 (也就是$N_{jk}$，但因為是方陣，為了code的簡化故這樣寫）

### join

把前面分開的$MN$矩陣分別丟進mapper1，map好之後，會得到$MN$分開的RDD，將這兩個RDD透過join的操作，即可所有擁有相同key值( j )，並且分別屬於$MN$兩個矩陣，會長得像 ( j , ((M, i, v1), (N, k, v2)))

### mapper2

在mapper2中，我根據上面join過後的資料結構，取出$M$矩陣的 i 值，$N$矩陣的 j 值，再將 v1 乘以 v2 得到這兩個位置相乘的value後，包成$( ( i, k ), P_{ikvalue})$ 的結構，回傳回去。

### reducer

在這裡，由於我們要將有相同key $(i,k)$的 value 全部相加起來，這樣才是$P_{ik}$的真正總和

故使用了reduceByKey，讓有相同Key值的value去執行我的reduce function

而我的reducer要做的事情，也就只是，傳進a, b兩個value後，兩個相加，回傳回去。

### Output

我原本的做法是直接collect起來後輸出，但經過測試後發現，若沒有自己去排序的話，會無法達到項要求要做的排序好的狀態

於是我使用RDD的 `sortByKey(ascending = True)` 的操作

再collect起來的話，就可以是排序後的樣子了。

最後，將上述的output用python的open 和 write即可寫進外部txt檔


### 參考網站
https://blog.csdn.net/helloxiaozhe/article/details/78481784

https://www.cnblogs.com/fang-jie/articles/6138789.html